In [1]:
import pandas as pd

/var/folders/nn/1yjvcwm12w78v3fpkk6_2_r00000gn/T/ipykernel_90704/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from spotify.spotify_service import SpotifyManager
import os

def choose_playlist(spotify_manager):
    playlists = spotify_manager.get_user_playlists()
    for i, playlist in enumerate(playlists['items']):
        print(f"{i+1}: {playlist['name']}")

    # choice = int(input("Enter the number of the playlist you want to analyze: "))
    choice=26
    return playlists['items'][choice - 1]['id']

def main():
    spotify_manager = SpotifyManager()
    playlist_id = choose_playlist(spotify_manager)
    full_track_data = spotify_manager.get_full_track_data(playlist_id)

    for track_data in full_track_data:
        track = track_data['track']
        print(f"{track['name']} by {track['artists'][0]['name']}")
        audio_features = track.get('audio_features')
        if audio_features:
            print(f"  - Tempo: {audio_features['tempo']}")
            print(f"  - Key: {audio_features['key']}")
            # Add more features as needed
    print(track_data)

In [3]:
spotify_manager = SpotifyManager()
playlist_id = choose_playlist(spotify_manager)
full_track_data = spotify_manager.get_full_track_data(playlist_id)

1: Facemelts Top 50 Songs of 23 (Official)
2: Bass
3: House Party
4: Home & Hosed
5: Something More
6: triple j's 40 Years of Music
7: Soft Office
8: Facemelts February 
9: ELETRONICAS 2024 ⚡ MAIS TOCADAS
10: Playlist #112 
11: Your Top Songs 2023
12: This Is Clutch
13: Paul Mcartney 2023
14: Bem
15: This Is Slightly Stoopid
16: Indie Halloween
17: Halloween 2023
18: Halloween Playlist 2024🎃
19: playlist #184
20: Funk Mix
21: Instrumental Rock
22: Garage band
23: This Is Pitty
24: This Is Bruno Mars
25: Foo fighters setlist 2023
26: Drums
27: Rock Titans
28: kilby girl energy
29: Funky Psychedelic Grooves
30: Parcels Discography
31: French
32: Foo Fighters 2023
33: Funky
34: New Be
35: Voice exercises 
36: Canto
37: 50s Party
38: New rock
39: GREATEST DRUMMERS OF ALL TIME
40: Ale cantar
41: Indie Rock Gems 2
42: Pagode Mix
43: Calm Indie Rock
44: Calm Alternative Rock
45: Calming Piano Instrumentals
46: 2000s Happy Road Trip
47: Hidden Gems of Rock
48: Indie Rock Hidden Gems
49: Noivad

In [4]:
full_track_data[0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [4]:
full_track_data[0]['track'].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri', 'audio_features', 'artist_info', 'album_info'])

In [5]:
full_track_data[0]['track']['album_info']['release_date']

'2013-03-08'

In [5]:
def filter_track_data(track_data):
    # Initialize a new dictionary to hold the filtered data
    filtered_data = {}

    # Extract track information
    track = track_data.get('track', {})
    filtered_data['song_name'] = track.get('name')
    filtered_data['song_popularity'] = track.get('popularity')

    # Extract audio features
    audio_features = track.get('audio_features', {})
    filtered_data['audio_features'] = {key: audio_features[key] for key in audio_features if key not in ['type', 'id', 'uri', 'track_href', 'analysis_url']}

    # Extract artist information (assuming first artist)
    artist_info = track.get('artist_info', {})
    filtered_data['artist_name'] = artist_info.get('name')
    filtered_data['artist_popularity'] = artist_info.get('popularity')
    # Include artist genres
    filtered_data['artist_genres'] = artist_info.get('genres', [])

    # Extract album information
    album_info = track.get('album_info', {})
    filtered_data['album_release_date'] = album_info.get('release_date')
    filtered_data['album_popularity'] = album_info.get('popularity')

    return filtered_data


In [6]:
def dict_to_dataframe(track_data):
    # Initialize a dictionary to hold the data for DataFrame creation
    df_data = {}

    # Add song name and artist name
    df_data['song_name'] = [track_data['song_name']]
    df_data['artist_name'] = [track_data['artist_name']]

    # Add numerical audio features
    for feature, value in track_data['audio_features'].items():
        df_data[feature] = [value]

    # Create a DataFrame
    df = pd.DataFrame(df_data)

    return df

In [7]:
for track_data in full_track_data:
    filtered_data = filter_track_data(track_data)
    df = dict_to_dataframe(filtered_data)
    print(df.shape)
    break

(1, 15)


In [8]:
df


,song_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,How To Fly,Sticky Fingers,0.677,0.644,8,-9.337,0,0.0971,0.161,0.00178,0.106,0.536,151.065,202187,4


In [16]:
filtered_data = filter_track_data(full_track_data[0])
filtered_data

{'song_name': 'How To Fly',
 'song_popularity': 71,
 'audio_features': {'danceability': 0.677,
  'energy': 0.644,
  'key': 8,
  'loudness': -9.337,
  'mode': 0,
  'speechiness': 0.0971,
  'acousticness': 0.161,
  'instrumentalness': 0.00178,
  'liveness': 0.106,
  'valence': 0.536,
  'tempo': 151.065,
  'duration_ms': 202187,
  'time_signature': 4},
 'artist_name': 'Sticky Fingers',
 'artist_popularity': 63.0,
 'artist_genres': ['australian reggae fusion',
  'australian surf rock',
  'reggae fusion'],
 'album_release_date': '2013-03-08',
 'album_popularity': 71}

In [15]:
def dict_to_dataframe(track_data):
    # Initialize a dictionary to hold the data for DataFrame creation
    df_data = {}

    # Add song name and artist name
    df_data['song_name'] = [track_data['song_name']]
    df_data['artist_name'] = [track_data['artist_name']]

    # Add numerical audio features
    for feature, value in track_data['audio_features'].items():
        df_data[feature] = [value]

    # Create a DataFrame
    df = pd.DataFrame(df_data)

    return df

In [17]:
dict_to_dataframe(filtered_data)

,song_name,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,How To Fly,Sticky Fingers,0.677,0.644,8,-9.337,0,0.0971,0.161,0.00178,0.106,0.536,151.065,202187,4


In [6]:
def print_dict_keys(d, indent=0):
    for key, value in d.items():
        print(' ' * indent + str(key))
        if isinstance(value, dict):
            print_dict_keys(value, indent + 4)  # Increase indentation for nested dictionaries


In [7]:
print_dict_keys(full_track_data[0])

added_at
added_by
    external_urls
        spotify
    href
    id
    type
    uri
is_local
primary_color
track
    album
        album_type
        artists
        available_markets
        external_urls
            spotify
        href
        id
        images
        name
        release_date
        release_date_precision
        total_tracks
        type
        uri
    artists
    available_markets
    disc_number
    duration_ms
    episode
    explicit
    external_ids
        isrc
    external_urls
        spotify
    href
    id
    is_local
    name
    popularity
    preview_url
    track
    track_number
    type
    uri
    audio_features
        danceability
        energy
        key
        loudness
        mode
        speechiness
        acousticness
        instrumentalness
        liveness
        valence
        tempo
        type
        id
        uri
        track_href
        analysis_url
        duration_ms
        time_signature
    artist_info
        ex